In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install pandas scikit-learn joblib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import joblib
import streamlit as st
from pyngrok import ngrok
import os


In [ ]:
import pandas as pd

# Load the CSV file
data_path = '/content/kidney_disease.csv'  # Replace with your CSV file path if different
ckd_data = pd.read_csv(data_path)

# Display the first few rows to understand the data structure
print(ckd_data.head())

# Check for null values and data types
print(ckd_data.info())
print(ckd_data.isnull().sum())


   id   age    bp     sg   al   su     rbc        pc         pcc          ba  \
0   0  48.0  80.0  1.020  1.0  0.0     NaN    normal  notpresent  notpresent   
1   1   7.0  50.0  1.020  4.0  0.0     NaN    normal  notpresent  notpresent   
2   2  62.0  80.0  1.010  2.0  3.0  normal    normal  notpresent  notpresent   
3   3  48.0  70.0  1.005  4.0  0.0  normal  abnormal     present  notpresent   
4   4  51.0  80.0  1.010  2.0  0.0  normal    normal  notpresent  notpresent   

   ...  pcv    wc   rc  htn   dm  cad appet   pe  ane classification  
0  ...   44  7800  5.2  yes  yes   no  good   no   no            ckd  
1  ...   38  6000  NaN   no   no   no  good   no   no            ckd  
2  ...   31  7500  NaN   no  yes   no  poor   no  yes            ckd  
3  ...   32  6700  3.9  yes   no   no  poor  yes  yes            ckd  
4  ...   35  7300  4.6   no   no   no  good   no   no            ckd  

[5 rows x 26 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 39

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Define binary categorical columns
binary_columns = ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane', 'classification']
label_encoders = {}

# Apply LabelEncoder to each binary column
for column in binary_columns:
    label_encoders[column] = LabelEncoder()
    ckd_data[column] = label_encoders[column].fit_transform(ckd_data[column])

# Check to confirm encoding
print(ckd_data.head())


   id   age    bp     sg   al   su  rbc  pc  pcc  ba  ...  pcv    wc   rc  \
0   0  48.0  80.0  1.020  1.0  0.0    2   1    0   0  ...   44  7800  5.2   
1   1   7.0  50.0  1.020  4.0  0.0    2   1    0   0  ...   38  6000  NaN   
2   2  62.0  80.0  1.010  2.0  3.0    1   1    0   0  ...   31  7500  NaN   
3   3  48.0  70.0  1.005  4.0  0.0    1   0    1   0  ...   32  6700  3.9   
4   4  51.0  80.0  1.010  2.0  0.0    1   1    0   0  ...   35  7300  4.6   

   htn  dm  cad appet pe ane  classification  
0    1   4    1     0  0   0               0  
1    0   3    1     0  0   0               0  
2    0   4    1     1  0   1               0  
3    1   3    1     1  1   1               0  
4    0   3    1     0  0   0               0  

[5 rows x 26 columns]


In [ ]:
# Fill missing values for numerical columns with the median
numerical_columns = ckd_data.select_dtypes(include=['float64', 'int64']).columns
for column in numerical_columns:
    ckd_data[column].fillna(ckd_data[column].median(), inplace=True)

# Convert object columns with numerical values to floats
for column in ['pcv', 'wc', 'rc']:
    ckd_data[column] = pd.to_numeric(ckd_data[column], errors='coerce')

# Fill missing values for these columns with the median after conversion
for column in ['pcv', 'wc', 'rc']:
    ckd_data[column].fillna(ckd_data[column].median(), inplace=True)

# Fill missing values for categorical columns with the mode
categorical_columns = ckd_data.select_dtypes(include='object').columns
for column in categorical_columns:
    ckd_data[column].fillna(ckd_data[column].mode()[0], inplace=True)

# Verify that there are no more missing values
print(ckd_data.isnull().sum())



id                0
age               0
bp                0
sg                0
al                0
su                0
rbc               0
pc                0
pcc               0
ba                0
bgr               0
bu                0
sc                0
sod               0
pot               0
hemo              0
pcv               0
wc                0
rc                0
htn               0
dm                0
cad               0
appet             0
pe                0
ane               0
classification    0
dtype: int64


<ipython-input-104-2f45914a3989>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ckd_data[column].fillna(ckd_data[column].median(), inplace=True)
<ipython-input-104-2f45914a3989>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpla

In [ ]:
from sklearn.preprocessing import StandardScaler

# List of columns to exclude from scaling
exclude_columns = ['id', 'age', 'rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane', 'classification']

# Identify numerical columns to scale (excluding the specified ones)
columns_to_scale = ckd_data.select_dtypes(include=['float64', 'int64']).columns.difference(exclude_columns)

# Initialize the StandardScaler
scaler = StandardScaler()

# Apply StandardScaler to the selected columns
ckd_data[columns_to_scale] = scaler.fit_transform(ckd_data[columns_to_scale])

# Check the scaled data
print(ckd_data.head())


   id   age        bp        sg        al        su  rbc  pc  pcc  ba  ...  \
0   0  48.0  0.254214  0.421486  0.076249 -0.380269    2   1    0   0  ...   
1   1   7.0 -1.972476  0.421486  2.363728 -0.380269    2   1    0   0  ...   
2   2  62.0  0.254214 -1.421074  0.838742  2.507853    1   1    0   0  ...   
3   3  48.0 -0.488016 -2.342354  2.363728 -0.380269    1   0    1   0  ...   
4   4  51.0  0.254214 -1.421074  0.838742 -0.380269    1   1    0   0  ...   

        pcv        wc        rc  htn  dm  cad  appet  pe  ane  classification  
0  0.603224 -0.197314  0.550044    1   4    1      0   0    0               0  
1 -0.132789 -0.909782  0.074073    0   3    1      0   0    0               0  
2 -0.991470 -0.316059  0.074073    0   4    1      1   0    1               0  
3 -0.868801 -0.632711 -0.996862    1   3    1      1   1    1               0  
4 -0.500795 -0.395222 -0.163913    0   3    1      0   0    0               0  

[5 rows x 26 columns]


In [ ]:
# Assuming 'classification' is the target variable
X = ckd_data.drop(columns=['classification'])
y = ckd_data['classification']


In [ ]:
# Encode categorical features
X = pd.get_dummies(X, drop_first=True)

In [ ]:
# Encode the target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train the Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
joblib.dump(model, 'ckd_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [ ]:
# Save the preprocessed data to a new CSV file
preprocessed_data_path = 'preprocessed_kidney_disease.csv'
ckd_data.to_csv(preprocessed_data_path, index=False)

print(f"Preprocessed data saved to {preprocessed_data_path}")

Preprocessed data saved to preprocessed_kidney_disease.csv


['id' 'age' 'bp' 'sg' 'al' 'su' 'rbc' 'pc' 'pcc' 'ba' 'bgr' 'bu' 'sc'
 'sod' 'pot' 'hemo' 'pcv' 'wc' 'rc' 'htn' 'dm' 'cad' 'appet' 'pe' 'ane']


In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import pandas as pd

# Load the trained model
model = joblib.load('ckd_model.pkl')

# Streamlit app title and introduction
st.title("Chronic Kidney Disease Prediction")
st.write("This application predicts whether a patient is at risk of chronic kidney disease based on input features.")

# Input fields for all required features
age = st.number_input("Age", min_value=0)
bp = st.number_input("Blood Pressure (bp)", min_value=0)
sg = st.number_input("Specific Gravity (sg)", min_value=0.0)
al = st.number_input("Albumin (al)", min_value=0)
su = st.number_input("Sugar (su)", min_value=0)
rbc = st.selectbox("Red Blood Cells (rbc)", ["Normal", "Abnormal"])
pc = st.selectbox("Pus Cell (pc)", ["Normal", "Abnormal"])
pcc = st.selectbox("Pus Cell Clumps (pcc)", ["Present", "Not Present"])
ba = st.selectbox("Bacteria (ba)", ["Present", "Not Present"])
bgr = st.number_input("Blood Glucose Random (bgr)", min_value=0)
bu = st.number_input("Blood Urea (bu)", min_value=0)
sc = st.number_input("Serum Creatinine (sc)", min_value=0.0)
sod = st.number_input("Sodium (sod)", min_value=0.0)
pot = st.number_input("Potassium (pot)", min_value=0.0)
hemo = st.number_input("Hemoglobin (hemo)", min_value=0.0)
pcv = st.number_input("Packed Cell Volume (pcv)", min_value=0)
wc = st.number_input("White Blood Cell Count (wc)", min_value=0)
rc = st.number_input("Red Blood Cell Count (rc)", min_value=0.0)
htn = st.selectbox("Hypertension (htn)", ["No", "Yes"])
dm = st.selectbox("Diabetes Mellitus (dm)", ["No", "Yes"])
cad = st.selectbox("Coronary Artery Disease (cad)", ["No", "Yes"])
appet = st.selectbox("Appetite (appet)", ["Good", "Poor"])
pe = st.selectbox("Pedal Edema (pe)", ["No", "Yes"])
ane = st.selectbox("Anemia (ane)", ["No", "Yes"])

# Convert categorical input to numerical
rbc = 1 if rbc == "Normal" else 0
pc = 1 if pc == "Normal" else 0
pcc = 1 if pcc == "Present" else 0
ba = 1 if ba == "Present" else 0
htn = 1 if htn == "Yes" else 0
dm = 1 if dm == "Yes" else 0
cad = 1 if cad == "Yes" else 0
appet = 1 if appet == "Good" else 0
pe = 1 if pe == "Yes" else 0
ane = 1 if ane == "Yes" else 0

# Prediction button
if st.button("Predict"):
    # Gather all inputs into a DataFrame, including 'id' with a placeholder value
    input_data = pd.DataFrame({
        'id': [0],  # Placeholder for 'id'
        'age': [age], 'bp': [bp], 'sg': [sg], 'al': [al], 'su': [su],
        'rbc': [rbc], 'pc': [pc], 'pcc': [pcc], 'ba': [ba],
        'bgr': [bgr], 'bu': [bu], 'sc': [sc], 'sod': [sod],
        'pot': [pot], 'hemo': [hemo], 'pcv': [pcv], 'wc': [wc],
        'rc': [rc], 'htn': [htn], 'dm': [dm], 'cad': [cad],
        'appet': [appet], 'pe': [pe], 'ane': [ane]
    })

    # Predict using the model
    prediction = model.predict(input_data)

    # Display the result
    if prediction[0] == 1:
        st.success("Predicted Class: Chronic Kidney Disease. The patient may have kidney issues.")
    else:
        st.success("Predicted Class: No Chronic Kidney Disease. The patient is likely healthy.")


Writing app.py


In [ ]:
!pkill -f ngrok


In [ ]:
!ps aux | grep ngrok


root       17140  0.0  0.0   7376  3572 ?        S    04:48   0:00 /bin/bash -c ps aux | grep ngrok
root       17142  0.0  0.0   6484  2276 ?        S    04:48   0:00 grep ngrok


In [ ]:
from pyngrok import ngrok
import os

# Install Streamlit and ngrok dependencies
os.system("pip install streamlit pyngrok")

# Start ngrok tunnel and specify the port explicitly
public_url = ngrok.connect(8501, "http")
print("ngrok tunnel URL:", public_url)

# Run Streamlit app (assuming app.py contains your Streamlit code)
os.system("streamlit run app.py &")


ngrok tunnel URL: NgrokTunnel: "https://00d7-34-44-103-15.ngrok-free.app" -> "http://localhost:8501"


0